# Activating Function Analysis

Analyze activation of neurite structured by DBS-imposed electric field

In [ ]:
import cPickle as pickle
import re, os

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
# Figure dimensions
fig_aspect_ratio = 8.0 / 3.0
page_width = 10
fig_width = 0.8 * page_width
fig_height = fig_width / fig_aspect_ratio
ax_width = 0.7 * page_width
ax_height = ax_width / fig_aspect_ratio

# Style of figures (default colors etc.): see https://matplotlib.org/gallery/style_sheets/style_sheets_reference.html
plt.style.use('default')

# Consistent size in backend 'notebook' (100 dpi) and 'inline' (72 dpi)
def plt_setrc():
    matplotlib.rcParams['savefig.dpi'] = 100
    matplotlib.rcParams['figure.dpi'] = 100

class bcolors:
    """ Example: print(bcolors.HEADER + 'hello' + bcolors.ENDC) """
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    ENDC = '\033[0m'

In [ ]:
%matplotlib notebook
plt_setrc()

# Import Data

In [ ]:
# SETPARAM: path to pickle files with population data
outputs = '/home/luye/Documents/sim_data/LuNetDBS/simdata_newsonic/sweeps_dbs-amp/sweep_dbs-amp_conf-V6_rec-axmid-EI/LuNetDBS_2019.09.03_16.31.28_job-44728_netconf-V6_rec-axmid-EI_dbs-amp-0.8/comp-locs_with-act-fun'

file_filter = lambda f: f.endswith('.pkl') and f.startswith('comp-locs')

filenames = os.listdir(outputs)
pkl_files = [os.path.join(outputs, f) for f in filenames if file_filter(f)]

# Structure
# 'STN' : {
#   'comp_act' : { 'somatic': list[float], 'basal': list[float], 'axonal': list[float]}
#   'comp_dists' : { 'somatic': list[float], 'basal': list[float], 'axonal': list[float]}
# }
#     
all_pop_data = {}
all_cell_offsets = {}

for pkl_fpath in pkl_files:
    pkl_fname = os.path.split(pkl_fpath)[1]
    matches = re.search(r'comp-locs_([\w\.]+)_', pkl_fpath)
    
    with open(pkl_fpath, 'rb') as f:
        data = pickle.load(f)
        
    if 'comp_act' not in data:
        print(bcolors.FAIL + "No activating function data: " + bcolors.ENDC + pkl_fname)
        continue
    else:
        print(bcolors.OKGREEN + "OK: activating function data found: " + bcolors.ENDC + pkl_fname)
    
    # Allocate data structure for population
    pop_label = data['population']
    all_pop_data[pop_label] = {}
    
    # Store some data as-is (per cell)
    for k in 'gids', 'comp_locs':
        all_pop_data[pop_label][k] = data[k]
    
    # Data with one dict per cell : flatten into single dict
    all_pop_data[pop_label]['comp_act'] = all_cells_act = data['comp_act'][0]
    all_pop_data[pop_label]['comp_dists'] = all_cells_dists = data['comp_dists'][0]
    
    # Store offset to data for each cell
    all_cell_offsets[pop_label] = {k: [0, len(vals)] for k, vals in all_cells_act.iteritems()}
    
    for i in range(1, len(data['comp_act'])):
        cell_act = data['comp_act'][i]
        cell_dists = data['comp_dists'][i]
        
        
        # Append this cell's data to population data
        for region in cell_act.keys():
            all_cells_act[region].extend(cell_act[region])
            all_cell_offsets[pop_label][region].append(
                all_cell_offsets[pop_label][region][-1] +  len(cell_act[region]))
            
        for region in cell_dists.keys():
            all_cells_dists[region].extend(cell_dists[region])

    # Convert to numpy array
    for region in all_cells_act.keys():
        all_cells_act[region] = np.array(all_cells_act[region])
    for region in all_cells_dists.keys():
        all_cells_dists[region] = np.array(all_cells_dists[region])

In [ ]:
def get_cell_dists_acts(pop, cell_gids, region):
    """
    Get activating function values and compartment distances for cell
    
    @return   dists, acts : tuple[numpy.array[float], numpy.array[float]]
              Distances and activating function values at compartment centers
    """
    cells_dists = []
    cells_acts = []
    for cell_gid in cell_gids:
        cell_index = all_pop_data[pop]['gids'].index(cell_gid)
        cell_data_start = all_cell_offsets[pop][region][cell_index]
        cell_data_stop = all_cell_offsets[pop][region][cell_index + 1]
        cell_slice = slice(cell_data_start, cell_data_stop)
        cells_acts.extend(all_pop_data[pop]['comp_act'][region][cell_slice])
        cells_dists.extend(all_pop_data[pop]['comp_dists'][region][cell_slice])
    return np.array(cells_dists), np.array(cells_acts)

# Scatter plots

In [ ]:
def plot_actfun_scatter(pop_label, multiple_figs=False, regions=None, cells='all'
                        regions_acts=None, regions_dists=None):
    """
    Scatter plot of activating function value vs distance from soma.
    """
    
    if regions is None:
        regions = all_pop_data[pop_label]['comp_act'].keys()
    
    if regions_acts is not None and regions_dists is not None:
        print("Caller supplied custon activation values")
    elif cells == 'all':
        regions_acts = all_pop_data[pop_label]['comp_act']
        regions_dists = all_pop_data[pop_label]['comp_dists']
    else:
        regions_dists_acts = {
            region: get_cell_dists_acts(pop_label, cells, region) for region in regions
        }
        regions_acts = {k: v[1] for k,v in regions_dists_acts.items()}
        regions_dists = {k: v[0] for k,v in regions_dists_acts.items()}
    
    if not multiple_figs:
        fig, ax = plt.subplots(figsize=(fig_width, ax_height))
        
    def decorate_fig(fig, ax, region=None):
        ax.legend()
        ax.grid()
        ax.set_xlabel('dist. from soma (um)')
        ax.set_ylabel('activating func. (mV/ms)')
        
        title = "Activating function"
        if region:
            title += " ({} compartments)".format(region)
        ax.set_title(title)
        fig.subplots_adjust(bottom=0.15) # prevent clipping of xlabel

    # Plot distance vs activating function
    for region in regions_acts.keys():
        if multiple_figs:
            fig, ax = plt.subplots(figsize=(fig_width, ax_height))
        
        ax.plot(regions_dists[region], regions_acts[region],
                '.', label=region, markersize=3)

        if multiple_figs:
            decorate_fig(fig, ax, region=region)
        
    if not multiple_figs:
        decorate_fig(fig, ax)

## STN

In [ ]:
plot_actfun_scatter('STN', multiple_figs=True)

## GPE

In [ ]:
plot_actfun_scatter('GPE.proto', multiple_figs=True)

# Boxplots

In [ ]:
def plot_actfun_boxplot(pop_label, regions=None, regions_acts=None, common_axis=False, **kwargs):
    
    if regions_acts is not None:
        regions = regions_acts.keys()
    elif regions is None:
        regions = all_pop_data[pop_label]['comp_act'].keys()
    
    if regions_acts is None:
        regions_acts = {region: all_pop_data[pop_label]['comp_act'][region] for region in regions}
        
    num_axes = 1 if common_axis else len(regions)
    fig, axes = plt.subplots(1, num_axes, figsize=(page_width, 1.5*ax_height))
    
    if common_axis:
        ax = axes
        x = [regions_acts[region] for region in regions] # one dataset per entry -> multiple boxplots
        bp = ax.boxplot(x, 0, 'g+', **kwargs)
        ax.grid(True, which='major', axis='y')
        ax.set_xticklabels(regions)
        ax.set_title("Activating function ({})".format(pop_label))
    else:
        # Stack axes horizontally
        for i, region in enumerate(regions):
            ax = axes[i]
            ax.set_title("Act. func. ({})".format(region))

            x = regions_acts[region]
            bp = ax.boxplot(x, 0, 'g+', **kwargs)
            ax.grid(True, which='major', axis='y')
            ax.set_xticks([])

            print("\nStatistics ({}):".format(region))
            print("- std : {:.3f}".format(np.std(x)))

## STN

In [ ]:
plot_actfun_boxplot('STN', showfliers=False)

## GPE

In [ ]:
plot_actfun_boxplot('GPE.proto', showfliers=True)

# Single cells

## Negative SEPs

In [ ]:
# STN cells with negative SEPs
pop_label = 'STN'
cell_gids = [6, 19, 21, 22, 23, 24, 25, 39, 40, 49, 9, 10]

# Get the data
regions = all_pop_data[pop_label]['comp_act'].keys()
regions_dists_acts = {
    region: get_cell_dists_acts(pop_label, cell_gids, region) for region in regions
}
regions_acts = {k: v[1] for k,v in regions_dists_acts.items()}
regions_dists = {k: v[0] for k,v in regions_dists_acts.items()}

# Filter axonal data
# ax_comp_inds = [i for i,d in enumerate(regions_dists['axonal']) if d < 500.0]
ax_comp_inds = [i for i,a in enumerate(regions_acts['axonal']) if a > 1000.0]
regions_acts['axonal'] = regions_acts['axonal'][ax_comp_inds]
regions_dists['axonal'] = regions_dists['axonal'][ax_comp_inds]

# Plot scatter and boxplot
plot_actfun_scatter(pop_label, multiple_figs=True, cells=cell_gids)
plot_actfun_boxplot(pop_label, regions_acts=regions_acts, showfliers=False)

## Positive SEPs

In [ ]:
# STN cells with positive SEPs
pop_label = 'STN'
cell_gids = [7, 8, 12, 13, 16, 18, 20, 26, 27, 28, 29, 30, 37, 43]

# Get the data
regions = all_pop_data[pop_label]['comp_act'].keys()
regions_dists_acts = {
    region: get_cell_dists_acts(pop_label, cell_gids, region) for region in regions
}
regions_acts = {k: v[1] for k,v in regions_dists_acts.items()}
regions_dists = {k: v[0] for k,v in regions_dists_acts.items()}

# Filter axonal data
# ax_comp_inds = [i for i,d in enumerate(regions_dists['axonal']) if d < 500.0]
ax_comp_inds = [i for i,a in enumerate(regions_acts['axonal']) if a > 1000.0]
regions_acts['axonal'] = regions_acts['axonal'][ax_comp_inds]
regions_dists['axonal'] = regions_dists['axonal'][ax_comp_inds]

# Plot scatter and boxplot
plot_actfun_scatter(pop_label, multiple_figs=True, cells=cell_gids)
plot_actfun_boxplot(pop_label, regions_acts=regions_acts, showfliers=False)